In [2]:
import numpy as np
import cv2
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img



# Load Caltech-101 dataset
# Adjust the path to your dataset directory

In [3]:

DATASET_PATH = "/kaggle/input/caltech-101/caltech-101"



# Define categories (folders in the dataset)

In [4]:

categories = [folder for folder in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, folder))]



# Parameters

In [5]:

IMAGE_SIZE = (224, 224)  # Required input size for VGG19
n_pca_components = 100  # Number of PCA components



# Handcrafted feature extractors

In [6]:

def extract_sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors



In [7]:
def extract_surf_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    surf = cv2.xfeatures2d.SURF_create()
    keypoints, descriptors = surf.detectAndCompute(gray, None)
    return descriptors



In [8]:
def extract_orb_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    return descriptors



# Load and preprocess images

In [9]:

def load_data():
    X_deep, X_handcrafted, y = [], [], []
    for idx, category in enumerate(categories):
        category_path = os.path.join(DATASET_PATH, category)
        for file in os.listdir(category_path):
            file_path = os.path.join(category_path, file)
            try:
                # Load image for deep learning features
                image = load_img(file_path, target_size=IMAGE_SIZE)
                image_array = img_to_array(image)
                image_array = preprocess_input(image_array)
                X_deep.append(image_array)

                # Load image for handcrafted features
                original_image = cv2.imread(file_path)
                sift_features = extract_sift_features(original_image)
                orb_features = extract_orb_features(original_image)

                # Combine handcrafted features (e.g., flatten and concatenate)
                combined_features = np.hstack([
                    np.mean(sift_features, axis=0) if sift_features is not None else np.zeros(128),
                    np.mean(orb_features, axis=0) if orb_features is not None else np.zeros(32)
                ])
                X_handcrafted.append(combined_features)

                y.append(idx)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    return np.array(X_deep), np.array(X_handcrafted), np.array(y)



# Load data

In [10]:

X_deep, X_handcrafted, y = load_data()



# Deep feature extraction using VGG19

In [11]:

vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
X_deep_features = vgg19_model.predict(X_deep)
X_deep_features = X_deep_features.reshape(X_deep_features.shape[0], -1)  # Flatten



286/286 ━━━━━━━━━━━━━━━━━━━━ 45s 156ms/step


# Reduce dimensionality of handcrafted features usin

In [12]:
# Reduce dimensionality of handcrafted features using PCA
pca = PCA(n_components=n_pca_components)
X_handcrafted_reduced = pca.fit_transform(X_handcrafted)



# Combine deep and handcrafted featur

In [13]:

X_combined = np.hstack([X_deep_features, X_handcrafted_reduced])



# Split data into training and testing sets

In [14]:

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)



# Train a Random Forest classifier

In [15]:

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

# Make predictions and evaluate

In [16]:

y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")


Classification Accuracy: 79.50%
